In [1]:
import pandas as pd

## Import data

In [60]:
df = pd.read_csv("./data/fake_or_real_news.csv")
df.rename(columns={df.columns[0] : "index"}, inplace=True)
df.set_index("index", inplace=True)
df.head()

,title,text,label
index,,,
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [65]:
df_train = pd.read_csv("./data/Data_latih.csv", index_col="ID")
df_train.head()

,label,tanggal,judul,narasi,nama file gambar
ID,,,,,
71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


In [73]:
df_train.isna().sum()

label               0
tanggal             0
judul               0
narasi              0
nama file gambar    0
dtype: int64

In [75]:
df_train["nama file gambar"].value_counts().max()

1

In [76]:
df_train["tanggal"].value_counts().max()

18

In [77]:
df_train.drop(columns=["nama file gambar"], inplace=True)

In [67]:
df_test = pd.read_csv("./data/Data_uji.csv", index_col="ID")

In [68]:
labels = df.label
labels.head()

index
8476     FAKE
10294    FAKE
3608     REAL
10142    FAKE
875      REAL
Name: label, dtype: object

In [10]:
target=df.label.value_counts()
target

label
REAL    3171
FAKE    3164
Name: count, dtype: int64

In [ ]:
# sns.countplot(df.label)
# plt.title('the number of news fake/real');

## Add Customize data

In [49]:
import requests
import googletrans
from bs4 import BeautifulSoup

In [ ]:
url = input("URL :  ")

raw_url = requests.get(url)
bs4 = BeautifulSoup(raw_url.content, "html.parser")
text = bs4.text

text

## Processing

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import more_itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [12]:
#default theme
plt.style.use('ggplot')
sns.color_palette("tab10")
sns.set(context='notebook', style='darkgrid', font='sans-serif', font_scale=1, rc=None)
matplotlib.rcParams['figure.figsize'] =[20,8]
matplotlib.rcParams.update({'font.size': 15})
matplotlib.rcParams['font.family'] = 'sans-serif'

In [25]:
X = df["text"]
y = df["label"]

x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((5068,), (1267,), (5068,), (1267,))

In [115]:
X = df_train["narasi"]
y = df_train["label"]

x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((3384,), (847,), (3384,), (847,))

In [134]:
#DataFlair - Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

#DataFlair - Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(df_train.narasi) 
tfidf_test=tfidf_vectorizer.transform(x_test.drop(columns=["tanggal", "nama file gambar"]))

In [135]:
y_train.shape, tfidf_train.shape

((3384,), (4231, 18481))

In [136]:
tfidf_test.shape

(847, 18481)

## Pipeline

In [99]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import 

In [104]:
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("tfidf", TfidfVectorizer()),
    ("Pac", PassiveAggressiveClassifier(n_jobs=-1))
])

In [113]:
pipeline.fit(df_train.drop(columns=["tanggal", "judul"]), df_train["label"].to_numpy().ravel())

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [114]:
df_train.drop(columns=["tanggal", "judul"])

,label,narasi
ID,,
71,1,A caller to a radio talk show recently shared ...
461,1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
495,1,Jokowi adalah presiden terbaik dlm sejarah ban...
550,1,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
681,1,Kadrun kalo lihat foto ini panas dingin . .
...,...,...
999287,1,KPK Dilarang Membawa Brimob Bersenjata Masuk G...
999472,1,Jangan mau dialihkan kepada pakaian adat… FOKU...
999501,1,Akhir nya bisa terkenal bang denny


In [137]:
#DataFlair - Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=150, verbose=1, n_jobs=-1, C=5)
pac.fit(tfidf_train,df_train.label)

#DataFlair - Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
#score=accuracy_score(y_test,y_pred)
#print(f'Accuracy: {round(score*100,2)}%')

-- Epoch 1
Norm: 42.93, NNZs: 15851, Bias: 0.688487, T: 4231, Avg. loss: 0.536497
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 53.25, NNZs: 16384, Bias: 0.750294, T: 8462, Avg. loss: 0.191517
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 58.41, NNZs: 16488, Bias: 0.773318, T: 12693, Avg. loss: 0.093555
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 61.24, NNZs: 16525, Bias: 0.780893, T: 16924, Avg. loss: 0.051867
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 62.93, NNZs: 16531, Bias: 0.781781, T: 21155, Avg. loss: 0.031031
Total training time: 0.02 seconds.
-- Epoch 6
Norm: 63.93, NNZs: 16531, Bias: 0.798029, T: 25386, Avg. loss: 0.020259
Total training time: 0.02 seconds.
-- Epoch 7
Norm: 64.62, NNZs: 16531, Bias: 0.805884, T: 29617, Avg. loss: 0.014999
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 65.16, NNZs: 16531, Bias: 0.795572, T: 33848, Avg. loss: 0.012028
Total training time: 0.02 seconds.
-- Epoch 9
Norm: 65.61, NNZs: 16531, Bias: 0.809298, T: 38

In [118]:
y_pred, y_pred.shape

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [129]:
test = [
"""
Jakarta Pusat, Kominfo – Wakil Presiden (Wapres) K.H. Ma’ruf Amin menilai pelayanan Haji oleh Pemerintah Arab Saudi terus meningkat dan semakin baik. Hal ini disampaikan Wapres kepada awak media Al Arabiya News Channel dalam tayangan video yang diunggah akun resmi Sekretariat Wakil Presiden pada Sabtu (08/06/2024).
“Pelayanan untuk jemaah kami cukup memperoleh perhatian. Walaupun saya mengatakan pada beliau [pejabat Arab Saudi] bahwa Indonesia seharusnya ditambah lagi [kuotanya] karena di daerah-daerah harus menunggu hingga 50 tahun,” ungkapnya.
Namun demikian, Mantan Ketua MUI ini, tetap menyampaikan rasa terima kasih dan penghargaan yang tinggi kepada Pemerintah Arab Saudi atas tambahan kuota haji yang diberikan setiap tahun, serta peningkatan kualitas pelayanan visa dan fasilitas di Arafah dan Mina.
“Saya menyampaikan apresiasi dan penghargaan kepada pemerintah Saudi, karena pemerintah Saudi pertama memberikan tambahan kuota untuk tahun ini dan juga telah menyiapkan pelayanan-pelayanan yang sangat baik, [termasuk] pelayanan pengurusan visa sehingga visa itu cepat sekali [keluarnya],” tuturnya.
Lebih lanjut, Wapres mengungkapkan bahwa semangat masyarakat Indonesia untuk beribadah haji sangat tinggi, bahkan bagi mereka yang pernah melaksanakan umroh. Namun sayangnya, di beberapa daerah masa tunggu haji sangat lama.
“Andaikan diberi tambahan, berarti akan mempercepat walaupun hanya beberapa tahun akan memanfaatkan kuota itu,” harapnya.
Lebih jauh pada kesempatan ini, Wapres menyampaikan berbagai pandangan terkait pelaksanaan haji, termasuk memuji koordinasi yang baik antara Pemerintah Indonesia dan Arab Saudi. Menurutnya, Menteri Haji Arab Saudi sendiri mengakui bahwa Indonesia termasuk negara dengan pengurusan administrasi yang sangat baik sehingga memudahkan koordinasi.
“Walaupun sudah baik, tetapi kita terus berusaha memperbaiki. Karena haji ini memindahkan orang dalam waktu yang singkat. Saya menyebutnya banyak muskilat, masalah baru. Dan itu terus kita usahakan supaya tidak ada di tahun depan,” tegasnya.
Selain itu, Wapres juga mengapresiasi kemajuan pesat pembangunan infrastruktur di Madinah guna mendukung pelaksanaan ibadah haji. Termasuk, perbaikan hotel dan pengaturan transportasi yang semakin modern dan dinamis.
“Saya kira pembangunan yang paling cepat dan terus berkembang di dunia ini, salah satunya di Madinah. Ada perbaikan-perbaikan, penambahan hotel kemudian pengaturan-pengaturan selalu ada perbaikan, hal ini memudahkan orang untuk berhaji sehingga tidak terjadi penumpukan,” ujarnya.
Terakhir, Wapres juga memuji peningkatan kapasitas lantai di Mina dan kelancaran transportasi di bandara baik di Jeddah maupun Madinah yang dapat menerima pesawat setiap lima menit.
“Melayani jemaah haji dalam jumlah besar, menurut saya bukanlah hal mudah, dan itu patut diapresiasi, dikasih acungan jempol, karena pelayanan yang terus berbenah menuju perbaikan-perbaikan setiap tahunnya,” pungkas Wapres.
""",
"""
Gunungkidul - Tanggul penahan gelombang di tempat pelelangan ikan (TPI) pantai Baron di Kapanewon Tanjungsari, Kabupaten Gunungkidul ambrol. Kabar ambrolnya tanggul tersebut sempat viral di media sosial.
Kabar ambrolnya tanggul diunggah media sosial Instagram oleh akun @updatedisini pada Jumat (31/5). Akun tersebut menarasikan tanggul ambrol usai dihantam gelombang pasang beberapa hari belakangan di pantai kawasan Gunungkidul.
"Detik Detik Tanggul TPI Pantai Baron Ambrol. Pasca gelombang pasang yang terjadi beberapa hari kemarin, Tanggul pengaman TPI (Tempat Pelelangan Ikan) ambruk.
Ambruknya tanggul tersebut diduga dikarenakan pasir penahan tanggul terkikis oleh air saat terjadi gelombang pasang. (31/05/2024) Pagi," tulis akun itu seperti dilihat detikJogja, Jumat (31/5/2024).
Dari video yang diunggah itu tampak tanggul perlahan roboh terjatuh ke laut. Hanya sebagian tanggul yang roboh dan sisanya masih berdiri.
Robohnya tanggul disaksikan oleh sejumlah warga yang berada di TPI Baron. Kemudian warga mengerumuni tanggul dan menyelamatkan beberapa benda 
Dimintai konfirmasi, Kepala Pelaksana Tugas BPBD Gunungkidul, Purwono membenarkan kejadian itu. Purwono mengatakan, tanggul roboh pada hari ini, Jumat (31/5). Beruntung tidak ada korban akibat robohnya tanggul tersebut.
"Kejadian sekitar 07.30 pagi tadi. Tidak ada korban," ungkap Purwono kepada wartawan melalui telepon, Jumat (31/5).
Purwono mengatakan tanggul ambrol akibat dampak dari gelombang tinggi yang terjadi di pantai di kawasan Gunungkidul. Adapun panjang tanggul yang ambrol berkisar 12 meter.
"Kejadian itu dampak gelombang tinggi. Tanggul kurang lebih 12 meter yang roboh. Total panjangnya 20 meter," katanya.
Tanggul tersebut, kata Purwono, berfungsi untuk melindungi TPI Baron dari ancaman gelombang tinggi dan air laut. Saat ini pihaknya masih menginventarisasi kerugian dan akan melaporkan ke Sekretaris Daerah (Sekda) Gunungkidul, Sri Suhartanta.
"Ini baru kita inventarisasi kerugian dan dampak kerusakan dan akan dilaporkan ke pimpinan, Pak Sekda," tuturnya.
Purwono menyebutkan tanggul tersebut dibangun sekitar tiga tahun lalu. Adapun material bangunannya berasal dari cor beton.

"Itu cor beton. Dibangun sekitar tiga tahun lalu,"pungkasnya.
"""
]




In [127]:
df_test_vec = tfidf_vectorizer.transform(df_test.narasi) # bentuknya harus list atau array
df_test_predict = pac.predict(df_test_vec)

In [128]:
df_test_predict

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,

In [138]:
df_test

,tanggal,judul,narasi,nama file gambar
ID,,,,
238057,13-Jul-20,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,238057.jpg
238158,6-Jul-20,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,238158.jpg
238865,22-Apr-20,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,238865.jpg
248298,22-Oct-19,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,248298.jpg
255176,1-May-20,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",255176.jpg
...,...,...,...,...
206021,1-May-20,"Dijemput oleh Tim Medis, Kakak Beradik Usia 8 ...","DiTangerang, dua bersaudara kakak beradik usia...",206021.jpg
219874,2-Oct-17,": Pertanda Alam, Kera Putih Mulai Turun Dari G...","Pertanda Alam, Kera Putih Mulai Turun Dari Gun...",219874.jpg
222802,22-Mar-20,Pasar Wameo dan Karya Nugraha di Tutup selama ...,Assalamu Alaikum …\n \n Info dari pedagang pa...,222802.jpg


In [132]:
def predict(texts):
    vec = tfidf_vectorizer.transform(texts) # bentuknya harus list atau array
    pred = pac.predict(vec)
    return pred

In [139]:
predict(test)

array([1, 1], dtype=int64)